In [4]:
!pip install pandas
!pip install nltk


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 4.9 MB/s  0:00:0036m-:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [nltk]4/5 [nltk]b]


In [50]:
!pip install scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 16.4 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.4/22.4 MB 29.5 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [scikit-learn] [scikit-learn]


In [55]:
import pandas as pd
import nltk

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense
from sklearn.model_selection import train_test_split


In [19]:
nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to /Users/tolan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /Users/tolan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [88]:
# Loading the dataset
neg_df = pd.read_csv("test_Arabic_tweets_negative_20190413.csv", header=None, names=["text"])
neg_df['label'] = 0

pos_df = pd.read_csv("test_Arabic_tweets_positive_20190413.csv", header=None, names=["text"])
pos_df['label'] = 1
df = pd.concat([neg_df, pos_df], ignore_index=True)
df = df.sample(frac=1).reset_index(drop=True)

df.head(10)


,text,label
0,ليله ستكتب عنها كل الأقلام ليله ستحصد في التار...,0
1,الى متى التعامل السئ للخادمات وعدم احترامهم وك...,0
2,هو اكيد شايف ومتذكر الاغلب 🤔 بس لو بمنشن ويرد ...,1
3,ماعندنا شك فيك يامعلم ونعلم مابداخلك من عشق لل...,1
4,رئيس المجلس العسكري السوداني الفريق الركن #عوض...,1
5,في مباراة العربية غوميز استفز وتلفظ على المداف...,1
6,اختي راحت للشاطي وافطرت وتقهوت بهذا الجو حلو و...,0
7,الله يشفيك ويريحنا من امثالك,0
8,ومن أحياها فكأنما أحيا الناس جميعا شابة (١٨سنة...,1
9,"جنابو زاتو محتار "" اتنين في يومين يجماعه"" 😂",1


In [89]:
#preprocessing text

stop_words = set(stopwords.words('arabic'))
texts = df['text'].astype(str).tolist()
labels = df['label'].values


In [110]:
vocab_size = 10000  # max vocabulary size
input_length = 100    # max length of sequences
embedding_dim = 100
tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post', truncating='post')

In [111]:

X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)


In [112]:
import numpy as np
print(X_train.shape)
print(np.isnan(X_train).any())
print(y_train[:10])
print(X_train.dtype)

(9216, 100)
False
[0 0 0 1 1 1 0 0 0 0]
int32


In [113]:
model=Sequential()
model.add(Embedding(input_dim=vocab_size,output_dim=embedding_dim,input_len=input_length))
model.add(LSTM(128))
model.add(Dense(1,activation='sigmoid'))
model.build(input_shape=(None,input_len))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

ValueError: Unrecognized keyword arguments passed to Embedding: {'input_len': 100}

In [61]:
model.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_test, y_test))


Epoch 1/5
710/710 ━━━━━━━━━━━━━━━━━━━━ 1897s 3s/step - accuracy: 0.0000e+00 - loss: nan - val_accuracy: 0.0000e+00 - val_loss: nan
Epoch 2/5
710/710 ━━━━━━━━━━━━━━━━━━━━ 73s 103ms/step - accuracy: 0.0000e+00 - loss: nan - val_accuracy: 0.0000e+00 - val_loss: nan
Epoch 3/5
710/710 ━━━━━━━━━━━━━━━━━━━━ 72s 101ms/step - accuracy: 0.0000e+00 - loss: nan - val_accuracy: 0.0000e+00 - val_loss: nan
Epoch 4/5
710/710 ━━━━━━━━━━━━━━━━━━━━ 67s 95ms/step - accuracy: 0.0000e+00 - loss: nan - val_accuracy: 0.0000e+00 - val_loss: nan
Epoch 5/5
710/710 ━━━━━━━━━━━━━━━━━━━━ 71s 100ms/step - accuracy: 0.0000e+00 - loss: nan - val_accuracy: 0.0000e+00 - val_loss: nan


In [105]:
loss,accuracy =model.evaluate(X_test,y_test)
print(f"test Accuracy:{accuracy:.2f}%".format(accuracy*100))

72/72 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 0.4770 - loss: 0.6933
test Accuracy:0.48%


In [106]:
def predict_sentiment(text):
    clean_text=preprocess_text(text)
    encoded_text=tokenizer.texts_to_sequences([clean_text])
    padded_text=pad_sequences(encoded_text,maxlen=max_length,padding="post")
    sentiment=model.predict(padded_text)
    return sentiment
    text="امطرت بالليل وانا نايمه؟ 😔"
    sentiment=predict_sentiment(text)
    print("Sentiment: {:.2f}".format(sentiment[0][0]))
